Montero Barraza Álvaro David

Tecnologías de lenguaje natural 5BV1

Ingeniería en IA

Este programa identifica frases, palabras y documentos similares, recibe 10 documentos para generar el corpus

Última modificación: 9/6/24



## **Modulos**

In [2]:
import spacy
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from copy import deepcopy
import re
import random
from keybert import KeyBERT
from collections import Counter
from itertools import product
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import transformers

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## **Funciones**

In [3]:
def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized

# Elimina etiquetas HTML y comentarios dentro del propio texto como indicaciones extra entre parentesis o comentarios entre --
def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-zA-Z\s.,]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
    
# Removemos stop words unicamente en ingles
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Lematizacion mediante spacy
nlp = spacy.load("en_core_web_trf")
def lemmatize(doc):
    docs = list(nlp.pipe([doc]))
    lemas = ' '.join([token.lemma_ for doc in docs for token in doc])
    return lemas



## **Análisis exploratorio y preprocesamiento**

In [4]:
Book_list=['/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/ancient_egypt.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/christ_roma.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/city_rome.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/early_european.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/EGYPT.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/greek_cities.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/greek_imp.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/Punic_war.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/religion_pagan.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/roman_religion.txt']
#Book_list=[]

Books_raw_file=[]
for i in range(len(Book_list)):
    with open(Book_list[i],'r',encoding='utf-8')as f:
        file=f.read()
        Books_raw_file.append(file)

In [7]:

if spacy.prefer_gpu():
    print("El modelo está configurado para usar GPU")
else:
    print("El modelo está configurado para usar CPU")

El modelo está configurado para usar CPU


In [13]:

nlp=spacy.load('en_core_web_trf')
spacy.prefer_gpu()
books=[]
books_w_stops=[]
books_w_process=[]
for i,book in enumerate(Books_raw_file):
    book=book.lower()
    book=eliminar_etiquetas(book)
    book=remover_apostrofes(book)
    book=remover_especiales(book)
    book=dobles_espacios(book)
    
    doc=nlp(book.lower())
    books_w_process.append(book)
    books_w_stops.append(doc.copy())#Libros con stopwords
    
    book=remove_stopwords(book)
    doc_s=nlp(book)
    
    books.append(doc_s.copy())#Libros sin stopwords
    print(f"Libro {i} {len(doc)} tokens  {len(list(doc.sents))} oraciones")

Libro 0 5590 tokens  159 oraciones
Libro 1 13995 tokens  436 oraciones
Libro 2 1200 tokens  59 oraciones
Libro 3 4270 tokens  222 oraciones
Libro 4 14396 tokens  427 oraciones
Libro 5 10223 tokens  300 oraciones
Libro 6 9298 tokens  288 oraciones
Libro 7 4234 tokens  170 oraciones
Libro 8 8224 tokens  269 oraciones
Libro 9 8213 tokens  267 oraciones


In [14]:
print("Sin stopwords")
for i,book in enumerate(books):
    print(f"Libro {i} {len(book)} tokens  {len(list(book.sents))} oraciones")

Sin stopwords
Libro 0 3212 tokens  180 oraciones
Libro 1 7905 tokens  508 oraciones
Libro 2 611 tokens  58 oraciones
Libro 3 2598 tokens  226 oraciones
Libro 4 7993 tokens  451 oraciones
Libro 5 5847 tokens  309 oraciones
Libro 6 5026 tokens  314 oraciones
Libro 7 2311 tokens  183 oraciones
Libro 8 4563 tokens  277 oraciones
Libro 9 4492 tokens  286 oraciones


## **Desarrollo de la práctica**

Similitud de palabras con synsets

In [15]:
for i,book in enumerate(books):
    print(book[10].pos_)

ADJ
ADJ
ADJ
ADJ
PROPN
NOUN
ADV
PROPN
ADJ
VERB


In [16]:
def most_common_verb(doc):
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    if not verbs:
        return None
    most_common = Counter(verbs).most_common(1)[0][0]
    return most_common

def most_common_verb_trf(doc):#Uso con transformers
    doc=nlp(doc)
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    if not verbs:
        return None
    most_common = Counter(verbs).most_common(1)[0][0]
    return most_common

def most_common_noun(doc):
    nouns = [token.lemma_ for token in doc if token.pos_=='NOUN']
    #print(len(nouns))
    if not nouns:
        print("No hubo")
        return None
    most_common = Counter(nouns).most_common(1)[0][0]
    return most_common


def most_common_terrm(doc):
    nouns = [token.lemma_ for token in doc if  token.pos_ !='PUNCT']
    #print(len(nouns))
    if not nouns:
        print("No hubo")
        return None
    most_common = Counter(nouns).most_common(1)[0][0]
    return most_common

def most_common_term_trf(doc):#Uso con transformers
    doc=nlp(doc)
    nouns = [token.lemma_ for token in doc if  token.pos_ !='PUNCT']
    #print(len(nouns))
    if not nouns:
        print("No hubo")
        return None
    most_common = Counter(nouns).most_common(1)[0][0]
    return most_common

def get_synsets(word, lang='eng'):
    return wn.synsets(word, lang=lang, pos=wn.VERB)

def find_similar_terms(target, tokens, similarity_func):
    target_synsets = get_synsets(target)
    if not target_synsets:
        return []
    target_synset = target_synsets[0]
    
    similarities = []
    for verb in tokens:
        verb_synsets = get_synsets(verb)
        if not verb_synsets:
            continue
        verb_synset = verb_synsets[0]
        similarity = similarity_func(target_synset, verb_synset)
        if similarity is not None:
            similarities.append((verb, similarity))
    
    # Ordenar por similitud y obtener los 5 más similares
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [verb for verb, _ in similarities[:5]]



## Similitud de documentos con Synsets

In [18]:
#Terminos similares
for i,book in enumerate(books):
    most_common=most_common_verb(book)
    tokens = [token.lemma_ for token in book]#Lista de los tokens
    tokens=list(set(tokens))#Eliminamos repetidos
    tokens.remove(most_common)#Removemos para que no se compara con sí mismo
    similar_verbs_wup = find_similar_terms(most_common, tokens, wn.wup_similarity)#Usando wup
    similar_verbs_path = find_similar_terms(most_common, tokens, wn.path_similarity)#Usando path
    print(f'Libro {i}')
    print(f"Term: {most_common}     wup:  {similar_verbs_wup}")
    print(f"Term: {most_common}     path: {similar_verbs_path}")
    
    
    
    
    

Libro 0
Term: give     wup:  ['love', 'fruit', 'prevent', 'indulge', 'bore']
Term: give     path: ['love', 'fruit', 'prevent', 'indulge', 'bore']
Libro 1
Term: find     wup:  ['love', 'own', 'fruit', 'determined', 'base']
Term: find     path: ['love', 'own', 'fruit', 'determined', 'base']
Libro 2
Term: see     wup:  ['use', 'understanding', 'get', 'begin', 'lost']
Term: see     path: ['hear', 'sense', 'use', 'understanding', 'get']
Libro 3
Term: find     wup:  ['use', 'want', 'love', 'include', 'gather']
Term: find     path: ['use', 'want', 'love', 'include', 'gather']
Libro 4
Term: find     wup:  ['love', 'base', 'ruin', 'prevent', 'exist']
Term: find     path: ['love', 'base', 'ruin', 'prevent', 'exist']
Libro 5
Term: find     wup:  ['love', 'base', 'ruin', 'prevent', 'indulge']
Term: find     path: ['love', 'base', 'ruin', 'prevent', 'indulge']
Libro 6
Term: make     wup:  ['do', 'love', 'own', 'lack', 'base']
Term: make     path: ['do', 'love', 'own', 'lack', 'base']
Libro 7
Term: 

In [113]:
#Verbos similares
for i,book in enumerate(books):
    most_common=most_common_verb(book)
    tokens = [token.lemma_ for token in book if token.pos_=='VERB']#Lista de los tokens
    tokens=list(set(tokens))#Eliminamos repetidos
    tokens.remove(most_common)#Removemos para que no se compara con sí mismo
    similar_verbs_wup = find_similar_terms(most_common, tokens, wn.wup_similarity)#Usando wup
    similar_verbs_path = find_similar_terms(most_common, tokens, wn.path_similarity)#Usando path
    print(f'Libro {i}')
    print(f"verb: {most_common}     wup:  {similar_verbs_wup}")
    print(f"verb: {most_common}     path: {similar_verbs_path}")
    

Libro 0
verb: give     wup:  ['get', 'arrive', 'precede', 'start', 'spread']
verb: give     path: ['get', 'arrive', 'precede', 'start', 'spread']
Libro 1
verb: find     wup:  ['shame', 'own', 'allow', 'create', 'study']
verb: find     path: ['shame', 'own', 'allow', 'create', 'study']
Libro 2
verb: see     wup:  ['give', 'get', 'keep', 'know', 'disappear']
verb: see     path: ['hear', 'give', 'get', 'keep', 'know']
Libro 3
verb: find     wup:  ['start', 'spread', 'study', 'vary', 'base']
verb: find     path: ['start', 'spread', 'study', 'vary', 'base']
Libro 4
verb: find     wup:  ['dominate', 'utilize', 'allow', 'end', 'study']
verb: find     path: ['dominate', 'utilize', 'allow', 'end', 'study']
Libro 5
verb: find     wup:  ['allow', 'meet', 'hold', 'connect', 'go']
verb: find     path: ['allow', 'meet', 'hold', 'connect', 'go']
Libro 6
verb: make     wup:  ['do', 'enunciate', 'get', 'dominate', 'own']
verb: make     path: ['do', 'enunciate', 'get', 'dominate', 'own']
Libro 7
verb: m

In [114]:
for i,book in enumerate(books):
    tokens = [token.lemma_ for token in book if token.pos_=='NOUN']
    print(f'libro {i}  {len(tokens)}')

libro 0  989
libro 1  1869
libro 2  194
libro 3  931
libro 4  2090
libro 5  1280
libro 6  1559
libro 7  680
libro 8  1318
libro 9  1326


In [115]:
#Sustantivo
for i,book in enumerate(books):
    most_common=most_common_noun(book)
    tokens = [token.lemma_ for token in book if token.pos_=='NOUN']#Lista de los tokens
    tokens=list(set(tokens))#Eliminamos repetidos
    tokens.remove(most_common)#Removemos para que no se compara con sí mismo
    similar_nouns_wup = find_similar_terms(most_common, tokens, wn.wup_similarity)#Usando wup
    similar_nouns_path = find_similar_terms(most_common, tokens, wn.path_similarity)#Usando path
    print(f'Libro {i}')
    print(f"Noun: {most_common}     wup:  {similar_nouns_wup}")
    print(f"Noun: {most_common}     path: {similar_nouns_path}")

Libro 0
Noun: land     wup:  ['end', 'hold', 'travel', 'spread', 'feature']
Noun: land     path: ['end', 'hold', 'travel', 'spread', 'feature']
Libro 1
Noun: name     wup:  ['call', 'title', 'understanding', 'stag', 'end']
Noun: name     path: ['call', 'title', 'understanding', 'stag', 'end']
Libro 2
Noun: man     wup:  ['understanding', 'end', 'fighting', 'lack', 'ruin']
Noun: man     path: ['understanding', 'end', 'fighting', 'lack', 'ruin']
Libro 3
Noun: illustration     wup:  []
Noun: illustration     path: []
Libro 4
Noun: period     wup:  []
Noun: period     path: []
Libro 5
Noun: foot     wup:  ['supply', 'portion', 'power', 'gate', 'bottom']
Noun: foot     path: ['supply', 'portion', 'power', 'hand', 'gate']
Libro 6
Noun: city     wup:  []
Noun: city     path: []
Libro 7
Noun: ship     wup:  ['meet', 'support', 'use', 'encounter', 'practice']
Noun: ship     path: ['herd', 'place', 'transport', 'meet', 'port']
Libro 8
Noun: religion     wup:  []
Noun: religion     path: []
Libro

## Similitud de documentos con Synsets

In [116]:
#Extracción de oraciones clave
key_phrases=[]
key_model=KeyBERT()
for book in books_w_process:#Libros con stopwords
    key_phrase=key_model.extract_keywords(book,keyphrase_ngram_range=(4,4),nr_candidates=1)
    key=key_phrase[0]
    key_phrases.append(key[0])
    print(key)
    

('shape egypt like lily', 0.6914)
('transformation rome pagan christian', 0.7486)
('civilized roman people ruled', 0.694)
('egyptians kind paper papyrus', 0.679)
('later age prehistoric egypt', 0.7529)
('turkom shepherds end asia', 0.61)
('policy imperialism deal greeks', 0.6834)
('regulus advanced carthage conquering', 0.7003)
('begin description ancient religions', 0.6764)
('cicero plancus despair war', 0.6379)


In [80]:
def max_similarity(synsets1, synsets2, similarity_func):
    max_sim = 0
    for synset1, synset2 in product(synsets1, synsets2):
        sim = similarity_func(synset1, synset2)
        if sim is not None and sim > max_sim:
            max_sim = sim
    return max_sim

def sentence_similarity(sentence1, sentence2, similarity_func):
    #lematizar las frases
    
    synsets1 = [get_synsets(token.lemma_) for token in sentence1 if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']]
    synsets2 = [get_synsets(token.lemma_) for token in sentence2 if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']]
    
    # Filtrar palabras sin synsets
    synsets1 = [syn for syn in synsets1 if syn]
    synsets2 = [syn for syn in synsets2 if syn]
    
    if not synsets1 or not synsets2:
        return 0

    # Calcular las similitudes entre los pares de synsets y promediar
    total_sim = 0
    count = 0
    for synset_group1 in synsets1:
        for synset_group2 in synsets2:
            sim = max_similarity(synset_group1, synset_group2, similarity_func)
            if sim:
                total_sim += sim
                count += 1
    
    return total_sim / count if count else 0

In [117]:
for phrase in key_phrases:
    phrase=nlp(phrase)
Prefered_phrase=key_phrases[8]#quitamos la 8va frase 
key_phrases.remove(Prefered_phrase)
print(Prefered_phrase)
for i,phrase in enumerate(key_phrases):
    path_sim = sentence_similarity(Prefered_phrase, phrase, wn.path_similarity)
    print(f"Con: {phrase} = {path_sim}")

begin description ancient religions
Con: shape egypt like lily = 0.25
Con: transformation rome pagan christian = 0
Con: civilized roman people ruled = 0.2611111111111111
Con: egyptians kind paper papyrus = 0.25
Con: later age prehistoric egypt = 0.25
Con: turkom shepherds end asia = 0.23809523809523808
Con: policy imperialism deal greeks = 0.3333333333333333
Con: regulus advanced carthage conquering = 0.29166666666666663
Con: cicero plancus despair war = 0.29166666666666663


## Similitud sintáctica de palabras y documentos

Embeddings GloVe

In [19]:
embeddings_dict = {}
with open("/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica4/glove.6B/glove.6B.200d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
print(f"{len(embeddings_dict)} vectores almacenados")

400000 vectores almacenados


In [21]:
def find_closest_embeddings(embedding,embedding_dict):#Embedding de la palabra y el diccionario que escogimos
    return sorted(embedding_dict.keys(), key=lambda word: spatial.distance.cosine(embedding_dict[word], embedding))

In [26]:
for i,book in enumerate(books):
    book=[token for token in book if not token.is_stop and not token.is_punct]
    most_common=most_common_verb(book)
    tokens = [token.lemma_ for token in book if not token.is_punct]#Lista de los tokens
    tokens=list(set(tokens))#Eliminamos repetidos
    if most_common in tokens:
        tokens.remove(most_common)#Removemos para que no se compara con sí mismo
    embeddings_list=[(token,embeddings_dict[token]) for token in tokens if token in embeddings_dict]#Solo ponemos los tokens que aparecen en el texto
    similar_terms=find_closest_embeddings(embeddings_dict[most_common],dict(embeddings_list))[:5]
    similar_terms=[(spatial.distance.cosine(embeddings_dict[token],embeddings_dict[most_common]) ,token)for token in similar_terms]
    print(f'Libro {i}')
    print(f"Noun: {most_common}     Closest:  {similar_terms}")


Libro 0
Noun: give     Closest:  [(0.1593226107299469, 'take'), (0.18839561748695555, 'make'), (0.23757285277657259, 'come'), (0.26393991767803815, 'go'), (0.279018728763182, 'allow')]
Libro 1
Noun: find     Closest:  [(0.22592145445329803, 'able'), (0.24292740111803524, 'make'), (0.2438174501181356, 'come'), (0.2535970465454649, 'know'), (0.2563319111105645, 'help')]
Libro 2
Noun: live     Closest:  [(0.4271273547596577, 'people'), (0.44265209369391645, 'come'), (0.457291898275211, 'rest'), (0.4807647972238607, 'life'), (0.48460927543866095, 'go')]
Libro 3
Noun: find     Closest:  [(0.22592145445329803, 'able'), (0.24292740111803524, 'make'), (0.2438174501181356, 'come'), (0.2535970465454649, 'know'), (0.2584960495438203, 'way')]
Libro 4
Noun: find     Closest:  [(0.22592145445329803, 'able'), (0.24292740111803524, 'make'), (0.2438174501181356, 'come'), (0.2535970465454649, 'know'), (0.2563319111105645, 'help')]
Libro 5
Noun: find     Closest:  [(0.22592145445329803, 'able'), (0.24292

Embeddings BERT


In [9]:
def cosine_similarity(vec1, vec2):
    vec2=vec2.T
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [11]:
import torch
model=transformers.BertModel.from_pretrained('bert-base-uncased')
tokenizer=transformers.BertTokenizer.from_pretrained('bert-base-uncased')
for i,book in enumerate(books_w_process):
    #book=[token for token in book if not token.is_stop and not token.is_punct]
    most_common=most_common_verb_trf(book)
    
    encoded_term=tokenizer(most_common,return_tensors='pt',truncation=True,max_length=512)
    #encoded_book=tokenizer(book,return_tensors='pt',truncation=True)
    
    with torch.no_grad():
        term_outputs = model(**encoded_term).last_hidden_state.mean(dim=1)
        #book_outputs = model(**encoded_book).last_hidden_state.mean(dim=1)
        
    similarities = []
    """
    for j,term in enumerate(book_outputs):
        #print(term_outputs.last_hidden_state[0].shape)
        #print(term.shape)
        similarity = torch.cosine_similarity(term_outputs,term)
        similarities.append((similarity,tokenizer.decode(term)))"""
        
    
    tokens = [token.text for token in books[i] if not token.is_punct]#Lista de palabras
    tokens=list(set(tokens))#Eliminamos repetidos
    if most_common in tokens:
        tokens.remove(most_common)#Removemos para que no se compara con sí mismo
    for term in tokens:
        term_aux_tsr=tokenizer(term,return_tensors='pt',truncation=True,max_length=512)#Tokenizamos cada palabra
        with torch.no_grad():
            term_aux_hidden=model(**term_aux_tsr).last_hidden_state.mean(dim=1)#Obtenemos el embedding
        similarity=torch.cosine_similarity(term_outputs,term_aux_hidden)#Calculamos similitud coseno
        similarities.append((similarity,term))
        
        
    #print(len(similarities))
    sorted_indices = sorted(similarities,key=lambda x:x[0],reverse=True)
    #similar_terms = [(tokenizer.decode(encoded_book['input_ids'][i]), similarities[i]) for i in sorted_indices[:5]]
    similar_terms=sorted_indices[:5]
    
    print(f'Libro {i}')
    print(f"Term: {most_common}     Closest:  {similar_terms}")

Libro 0
Term: give     Closest:  [(tensor([0.9558]), 'giving'), (tensor([0.9352]), 'take'), (tensor([0.9199]), 'tell'), (tensor([0.9182]), 'make'), (tensor([0.9171]), 'spread')]
Libro 1
Term: find     Closest:  [(tensor([0.9009]), 'finding'), (tensor([0.8920]), 'searching'), (tensor([0.8894]), 'go'), (tensor([0.8893]), 'read'), (tensor([0.8878]), 'help')]
Libro 2
Term: see     Closest:  [(tensor([0.9091]), 'explains'), (tensor([0.9030]), 'respects'), (tensor([0.8977]), 'understand'), (tensor([0.8935]), 'swept'), (tensor([0.8929]), 'due')]
Libro 3
Term: find     Closest:  [(tensor([0.9081]), 'reach'), (tensor([0.8894]), 'go'), (tensor([0.8847]), 'learn'), (tensor([0.8788]), 'take'), (tensor([0.8770]), 'give')]
Libro 4
Term: find     Closest:  [(tensor([0.9081]), 'reach'), (tensor([0.9025]), 'hunt'), (tensor([0.8905]), 'seek'), (tensor([0.8894]), 'go'), (tensor([0.8893]), 'read')]
Libro 5
Term: find     Closest:  [(tensor([0.9081]), 'reach'), (tensor([0.8976]), 'save'), (tensor([0.8878])